In [1]:
import pandas as pd
import json

In [2]:
import numpy as np

In [3]:
wm_all = pd.read_parquet('weather_messages_2014-18.parquet')

In [4]:
wm_all

,time,v1,v2,v3
0,201401010000,"[""It's the early morning of a day in January. ...","[""It's the early morning of a day in January. ...","[""It's the early morning of a day in January. ..."
1,201401010600,"[""It's the morning of a day in January. "", ""Th...","[""It's the morning of a day in January. "", ""Th...","[""It's the morning of a day in January. "", ""Th..."
2,201401011200,"[""It's the afternoon of a day in January."", ""T...","[""It's the afternoon of a day in January."", ""T...","[""It's the afternoon of a day in January."", ""T..."
3,201401011800,"[""It's the evening of a day in January. "", ""Th...","[""It's the evening of a day in January."", ""The...","[""It's the evening of a day in January. "", ""T..."
4,201401020000,"[""It's the early morning of a day in January.""...","[""It's the early morning of a day in January.""...","[""It's the early morning of a day in January. ..."
...,...,...,...,...
7299,201812301800,"[""It's the evening of a day in December. "", ""T...","[""It's the evening of a day in December."", ""Th...","[""It's the evening of a day in December. "", ""T..."
7300,201812310000,"[""It's the early morning of a day in December....","[""It's the early morning of a day in December....","[""It's the early morning of a day in December...."
7301,201812310600,"[""It's the morning of a day in December. "", ""T...","[""It's the morning of a day in December. "", ""T...","[""It's the morning of a day in December."", ""Th..."
7302,201812311200,"[""It's the afternoon of the last day in Decemb...","[""It's the afternoon of a day in December."", ""...","[""It's the afternoon of a day in December."", ""..."


In [5]:
# find all the values that does not start with '["It's the'
wm_all[~wm_all['v1'].str.startswith('["It')]


,time,v1,v2,v3


In [6]:
wm_all[~wm_all['v2'].str.startswith('["It')]

,time,v1,v2,v3


In [7]:
wm_all[~wm_all['v3'].str.startswith('["It')]

,time,v1,v2,v3


In [8]:
# get all the values of 'v1', 'v2', 'v3' column
v1=list(wm_all['v1'].unique())
v2=list(wm_all['v2'].unique())
v3=list(wm_all['v3'].unique())


In [9]:
v1

['["It\'s the early morning of a day in January. ", "The current weather is clear. ", "The weather is expected to remain clear throughout. ", "The temperature is showing a slight increase. ", "There is Light Breeze from the south-southwest. ", "The atmospheric pressure is average. ", "The humidity levels are dropping but remain high."]',
 '["It\'s the morning of a day in January. ", "The current weather is partly sunny. ", "The weather is expected to clear up initially, becoming cloudier as the hours pass. ", "The temperature will see a slight drop before stabilizing. ", "There is a Gentle Breeze from the SW. ", "The atmospheric pressure is average. ", "The humidity will rise slightly, making the air quite humid."]',
 '["It\'s the afternoon of a day in January.", "The current weather has broken clouds.", "The weather is transitioning from partly sunny to passing clouds.", "The temperature is showing a slight decrease.", "There is a Gentle Breeze from S.", "The atmospheric pressure is a

In [10]:
len(v1)

7298

In [11]:
all_msgs=[]
all_striped_msgs=set()
for i in v1:
    messages = json.loads(i)
    for m in messages:
        m_striped = m.replace(' ','')
        # chcek if the m_striped in v1_strip
        if m_striped not in all_striped_msgs:
            all_msgs.append(m)
            all_striped_msgs.add(m_striped)
for i in v2:
    messages = json.loads(i)
    for m in messages:
        m_striped = m.replace(' ','')
        # chcek if the m_striped in v1_strip
        if m_striped not in all_striped_msgs:
            all_msgs.append(m)
            all_striped_msgs.add(m_striped)
for i in v3:
    messages = json.loads(i)
    for m in messages:
        m_striped = m.replace(' ','')
        # chcek if the m_striped in v1_strip
        if m_striped not in all_striped_msgs:
            all_msgs.append(m)
            all_striped_msgs.add(m_striped)

In [12]:
len(all_msgs)

33851

In [13]:
'' in all_msgs

True

In [14]:
all_msgs.remove('')

In [15]:
BERT_model= 'all-mpnet-base-v2'#'paraphrase-MiniLM-L6-v2' #'all-mpnet-base-v2'
import os
# save_dir='./Elec_news-embedding-'+BERT_model
# if not os.path.exists(save_dir):
#     os.makedirs(save_dir)
from sentence_transformers import SentenceTransformer
model = SentenceTransformer(BERT_model).to('cuda:0')
from tqdm import tqdm

In [16]:
model.encode(['fuckyou','fuckme']).shape

(2, 768)

In [17]:
import hashlib
# create file name
def get_file_name(sentence):
    h = hashlib.blake2b(digest_size=5)
    h.update(sentence.encode('utf-8'))
    return h.hexdigest()

In [18]:
get_file_name('fuck you')

'347f92efaf'

In [19]:
get_file_name("It's the early morning of a day in January. ".replace(' ',''))

'6e6f1f21d1'

In [20]:
hash_dict={}
for s in all_msgs:
    hash = get_file_name(s.replace(' ',''))
    hash_dict[s]=hash

In [21]:
hash_dict

{"It's the early morning of a day in January. ": '6e6f1f21d1',
 'The current weather is clear. ': '47bf306cf2',
 'The weather is expected to remain clear throughout. ': '934b56862a',
 'The temperature is showing a slight increase. ': 'b12c0f596f',
 'There is Light Breeze from the south-southwest. ': '9d144884dd',
 'The atmospheric pressure is average. ': '42474f0c90',
 'The humidity levels are dropping but remain high.': 'c6bd254ef5',
 "It's the morning of a day in January. ": '006b61dce8',
 'The current weather is partly sunny. ': '4ddefeaa3c',
 'The weather is expected to clear up initially, becoming cloudier as the hours pass. ': '33347dec56',
 'The temperature will see a slight drop before stabilizing. ': 'c6ba22deba',
 'There is a Gentle Breeze from the SW. ': '9b815d6ea5',
 'The humidity will rise slightly, making the air quite humid.': 'bac42158ca',
 "It's the afternoon of a day in January.": '5533f60c82',
 'The current weather has broken clouds.': 'fdcc8a09b0',
 'The weather is

In [24]:
hash_dict={}
hashes=set()
for i in range(0, len(all_msgs), 1000):
    
    send_msgs=all_msgs[i:i+1000]
    response = model.encode(send_msgs)
    print(f'doing {i}')
    for msg, emb in zip(send_msgs, response):
        name = get_file_name(msg.replace(' ',''))
        emb_array = emb
        # save the np array
        np.save(f'./{BERT_model}_caption_emb/{name}.npy', emb_array)
        hashes.add(name)
        hash_dict[msg] = name
    
    # with open('./hashtable.json', 'w') as f:
    #     json.dump(hash_dict,f)
    

doing 0
doing 1000
doing 2000
doing 3000
doing 4000
doing 5000
doing 6000
doing 7000
doing 8000
doing 9000
doing 10000
doing 11000
doing 12000
doing 13000
doing 14000
doing 15000
doing 16000
doing 17000
doing 18000
doing 19000
doing 20000
doing 21000
doing 22000
doing 23000
doing 24000
doing 25000
doing 26000
doing 27000
doing 28000
doing 29000
doing 30000
doing 31000
doing 32000
doing 33000


In [25]:
len(hashes)

33850

In [20]:
with open('./caption_emb/hashtable.json', 'w') as f:
    json.dump(hash_dict,f)

In [26]:
a='''p (mbar): Atmospheric pressure measured in millibars. It indicates the weight of the air above the point of measurement.

T (degC): Temperature at the point of observation, measured in degrees Celsius.

Tpot (K): Potential temperature, given in Kelvin. This is the temperature that a parcel of air would have if it were brought adiabatically to a standard reference pressure, often used to compare temperatures at different pressures in a thermodynamically consistent way.

Tdew (degC): Dew point temperature in degrees Celsius. It's the temperature to which air must be cooled, at constant pressure and water vapor content, for saturation to occur. A lower dew point means dryer air.

rh (%): Relative humidity, expressed as a percentage. It measures the amount of moisture in the air relative to the maximum amount of moisture the air can hold at that temperature.

VPmax (mbar): Maximum vapor pressure, in millibars. It represents the maximum amount of moisture that the air can hold at a given temperature.

VPact (mbar): Actual vapor pressure, in millibars. It's the current amount of water vapor present in the air.

VPdef (mbar): Vapor pressure deficit, in millibars. The difference between the maximum vapor pressure and the actual vapor pressure; it indicates how much more moisture the air can hold before saturation.

sh (g/kg): Specific humidity, the mass of water vapor in a given mass of air, including the water vapor. It's measured in grams of water vapor per kilogram of air.

H2OC (mmol/mol): Water vapor concentration, expressed in millimoles of water per mole of air. It's another way to quantify the amount of moisture in the air.

rho (g/m³): Air density, measured in grams per cubic meter. It indicates the mass of air in a given volume and varies with temperature, pressure, and moisture content.

wv (m/s): Wind velocity, the speed of the wind measured in meters per second.

max. wv (m/s): Maximum wind velocity observed in the given time period, measured in meters per second.

wd (deg): Wind direction, in degrees from true north. This indicates the direction from which the wind is coming.

rain (mm): Rainfall amount, measured in millimeters. It indicates how much rain has fallen during the observation period.

raining (s): Duration of rainfall, measured in seconds. It specifies how long it has rained during the observation period.

SWDR (W/m²): Shortwave Downward Radiation, the amount of solar radiation reaching the ground, measured in watts per square meter.

PAR (μmol/m²/s): Photosynthetically Active Radiation, the amount of light available for photosynthesis, measured in micromoles of photons per square meter per second.

max. PAR (μmol/m²/s): Maximum Photosynthetically Active Radiation observed in the given time period, indicating the peak light availability for photosynthesis.

Tlog (degC): Likely a logged temperature measurement in degrees Celsius. It could be a specific type of temperature measurement or recording method used in the dataset.

CO2 (ppm): Carbon dioxide concentration in the air, measured in parts per million. It's a key greenhouse gas and indicator of air quality.'''

In [27]:
a=a.split('\n\n')
a

['p (mbar): Atmospheric pressure measured in millibars. It indicates the weight of the air above the point of measurement.',
 'T (degC): Temperature at the point of observation, measured in degrees Celsius.',
 'Tpot (K): Potential temperature, given in Kelvin. This is the temperature that a parcel of air would have if it were brought adiabatically to a standard reference pressure, often used to compare temperatures at different pressures in a thermodynamically consistent way.',
 "Tdew (degC): Dew point temperature in degrees Celsius. It's the temperature to which air must be cooled, at constant pressure and water vapor content, for saturation to occur. A lower dew point means dryer air.",
 'rh (%): Relative humidity, expressed as a percentage. It measures the amount of moisture in the air relative to the maximum amount of moisture the air can hold at that temperature.',
 'VPmax (mbar): Maximum vapor pressure, in millibars. It represents the maximum amount of moisture that the air can h

In [28]:
response = model.encode(a)

In [29]:
for i in range(len(a)):
    emb_array = response[i]
    np.save(f'./{BERT_model}_caption_emb/channel_{i}.npy', emb_array)